In [1]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

import coco
import utils
import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "mylogs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco_humanpose.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
#COCO_DIR = "D:/Github/FastMaskRCNN/data/coco"  # TODO: enter value here
#IMAGE_DIR = os.path.join(ROOT_DIR, "images")
COCO_DIR = "E:/DSLR/01_Privat/2020/Hochzeit/Daniel/Fotos_Reportage"  # TODO: enter value here
IMAGE_DIR = COCO_DIR

In [2]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    KEYPOINT_MASK_POOL_SIZE = 7

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights

model_path = os.path.join(ROOT_DIR, "mask_rcnn_coco_humanpose.h5")
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(1, 1000, 4)
(None, None, 2)
(None, None, 2, 4)
(None, None)


ValueError: in converted code:

    C:\Users\LStue\Mask_RCNN_Humanpose\model.py:1074 call  *
        outputs = utils.batch_slice(
    C:\Users\LStue\Mask_RCNN_Humanpose\utils.py:994 batch_slice  *
        output_slice = graph_fn(*inputs_slice)
    C:\Users\LStue\AppData\Local\Temp\tmpw8yt_v14.py:15 None  *
        outputs = ag__.converted_call(utils.batch_slice, ([rois, mrcnn_class, mrcnn_bbox, window], lambda x, y, w, z: ag__.converted_call(refine_detections_graph, (x, y, w, z, self.config), None, fscope), self.config.IMAGES_PER_GPU), None, fscope)
    C:\Users\LStue\Mask_RCNN_Humanpose\model.py:959 refine_detections_graph  *
        indices = tf.stack([tf.range(probs.shape[0]), class_ids], axis=1)
    D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\tensorflow_core\python\ops\math_ops.py:1430 range
        limit = ops.convert_to_tensor(limit, dtype=dtype, name="limit")
    D:\Users\LStue\anaconda3\envs\tf23\lib\site-packages\tensorflow_core\python\framework\ops.py:1314 convert_to

In [ ]:
import cv2
# COCO Class names
#For human pose task We just use "BG" and "person"
class_names = ['BG', 'person']
# Load a random image from the images folder
file_names = next(os.walk(IMAGE_DIR))[2]
# image = skimage.io.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
image = cv2.imread(os.path.join(IMAGE_DIR, random.choice(file_names)))
#BGR->RGB
image = image[:,:,::-1]

# Run detection
results = model.detect_keypoint([image], verbose=1)
r = results[0] # for one image

log("rois",r['rois'])
log("keypoints",r['keypoints'])
log("class_ids",r['class_ids'])
log("keypoints",r['keypoints'])
log("masks",r['masks'])
log("scores",r['scores'])

visualize.display_keypoints(image,r['rois'],r['keypoints'],r['class_ids'],class_names,skeleton = inference_config.LIMBS)
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])